In [77]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as BS

In [78]:
# необходимые списки
titles = []
kk_list = []
B_list = []
G_list = []
U_list = []

In [79]:
headers = {
    'Accept': '*/*',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 YaBrowser/23.11.0.0 Safari/537.36'
}

# делаем get запрос gj url, и создаем объект soup
url = 'https://health-diet.ru/table_calorie/'
get_req = requests.get(url, headers=headers).text
soup =  BS(get_req, 'lxml')

# находим все повторяющиеся теги, в которых представленны
# ссылки на категории продуктов и делаем get запрос уже по ним
dish_groups = soup.find_all('div', class_='uk-flex mzr-tc-group-item')
for group in dish_groups:
    new_url = 'https://health-diet.ru' + group.find('a', href=True)['href']
    new_get_req = requests.get(new_url, headers=headers).text
    new_soup = BS(new_get_req, 'lxml')

    try:
        # собираем основную информацию о продуктах
        dishes = new_soup.find('tbody').find_all('tr')
        for one in dishes:
            title = one.find('td').find('a').text
            kk = one.find('td').find_next().text
            B = one.find('td').find_next().find_next().text
            G = one.find('td').find_next().find_next().find_next().text
            U = one.find('td').find_next().find_next().find_next().find_next().text

            titles.append(title)
            kk_list.append(kk)
            B_list.append(B)
            G_list.append(G)
            U_list.append(U)

    except AttributeError:
        continue

# составляем датафрейм из полученной информации
df = pd.DataFrame({
    "название": titles,
    "калорийность": kk_list,
    "белки": B_list,
    "жири": G_list,
    "углеводы" : U_list
})
display(df.head())
df.info()

,название,калорийность,белки,жири,углеводы
0,Антилопа,Антилопа,114 кКал,"22,38 г","2,03 г"
1,"Антилопа, запеченная","Антилопа, запеченная",150 кКал,"29,45 г","2,67 г"
2,Баранина 1 кат.,Баранина 1 кат.,209 кКал,"15,6 г","16,3 г"
3,Баранина 2 кат.,Баранина 2 кат.,166 кКал,"19,8 г","9,6 г"
4,"Баранина австралийская ,голяшка, мясо и жир, о...","Баранина австралийская ,голяшка, мясо и жир, о...",231 кКал,"25,25 г","13,69 г"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13019 entries, 0 to 13018
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   название      13019 non-null  object
 1   калорийность  13019 non-null  object
 2   белки         13019 non-null  object
 3   жири          13019 non-null  object
 4   углеводы      13019 non-null  object
dtypes: object(5)
memory usage: 508.7+ KB


In [80]:
df.tail()

,название,калорийность,белки,жири,углеводы
13014,Шоколадный Батончик Импульс [Яшкино],Шоколадный Батончик Импульс [Яшкино],485 кКал,"7,2 г","24,3 г"
13015,Шоколадный Батончик с Грушевой Начинкой [Яшкино],Шоколадный Батончик с Грушевой Начинкой [Яшкино],420 кКал,3 г,17 г
13016,Щербет с Изюмом и Арахисом [Яшкино],Щербет с Изюмом и Арахисом [Яшкино],480 кКал,6 г,9 г
13017,Ярче Семечки в Мягкой Карамели [Яшкино],Ярче Семечки в Мягкой Карамели [Яшкино],497 кКал,"9,6 г","30,5 г"
13018,Яшкинская Картошка [Яшкино],Яшкинская Картошка [Яшкино],537 кКал,"6,3 г","31,7 г"
